In [ ]:
# Cython imports/loads
%load_ext line_profiler
%load_ext Cython
import line_profiler
from Cython.Compiler import Options 
dd = Options.get_directive_defaults()
dd['linetrace'] = True # triggers profile=True
dd['binding'] = True

In [ ]:
# library imports
import sys
import numpy as np

In [ ]:
# nPoRe imports
sys.path.append(f'/home/timdunn/npore/src')
from aln import *
import cfg as cfg

In [ ]:
# set globals
class Globals():
    def __init__(self):
        self.max_n = 6
        self.max_l = 100
    
cfg.args = Globals()

In [ ]:
# %%cython -a -f --compile-args=-DCYTHON_TRACE_NOGIL=1
# import numpy, math, cython

# @cython.boundscheck(False)
# @cython.wraparound(False)
# @cython.cdivision(True)
# cpdef int[:,::1] get_np_info(char[::1] seq):
#     ''' Calculate N-polymer information. 
#          seq:     A T A T A T T T T T T T A A A G C
#          np_info:
#          RPTS:    3 3 3 3 3 7 7 7 7 7 7 7 3 3 3 0 0
#          RPT = 0-based index of the current repeat                                                         
#          N = number of bases in repeated sequence                                                          
#          IDX = 0-based index of current base within repeat                                                 
                                                                                                           
#          Explanation:                                                                                      
#          3(AT), 7(T), 3(A) with some overlap. A sequence must repeat at least                              
#          twice to be considered an n-polymer. Bases are considered part of                                 
#          the longest repeat in which they're included (N * RPTS).                                          
                                                                                                           
#     '''                                                                                                    
                                                                                                           
#     cdef int seq_len = len(seq)                                                                            
#     np_info_buf = numpy.zeros((4, seq_len), dtype=numpy.intc)                                                    
#     cdef int[:,::1] np_info = np_info_buf                                                                  
#     cdef int n, np_repeat_len, pos, rpt, idx                                                               
#     cdef int seq_idx, seq_ptr                                                                              
                                                                                                           
#     # define constant values for indexing into `np_info` array                                             
#     cdef int RPTS = 0                                                                                      
#     cdef int RPT = 1                                                                                       
#     cdef int N = 2                                                                                         
#     cdef int IDX = 3                         
#     max_np = 10
                                                                                                           
#     for seq_idx in range(seq_len): # iterate over sequence                                                 
                                                                                                           
#         best_len = 0                                                                                       
#         for n in range(1, max_np+1): # check each length N-polymer                                
                                                                                                           
#             # get np repeat length at this position                                                        
#             np_repeat_len = 0                                                                              
#             seq_ptr = seq_idx                                                                              
#             while seq_ptr+n < seq_len and seq[seq_ptr] == seq[seq_ptr+n]:                                  
#                 seq_ptr += 1                                                                               
#                 if (seq_ptr-seq_idx) % n == 0: # finished n-polymer                                        
#                     np_repeat_len += 1                                                                     
#             if np_repeat_len: np_repeat_len += 1 # count first                                             
                                                                                                           
#             # save n-polymer info                                                                          
#             if np_repeat_len > 2 and \
#                     n * np_repeat_len > np_info[N, seq_idx] * np_info[RPTS, seq_idx]:                      
#                 for rpt in range(np_repeat_len):                                                           
#                     for idx in range(n):                                                                   
#                         pos = seq_idx + rpt*n + idx                                                        
#                         np_info[RPTS, pos] = np_repeat_len                                                 
#                         np_info[RPT, pos] = rpt                                                            
#                         np_info[N, pos] = n                                                                
#                         np_info[IDX, pos] = idx                                                            
                                                                                                           
#     return np_info   



# @cython.boundscheck(False)
# @cython.wraparound(False)
# cdef float np_score(int n, int ref_np_len, int indel_len, float[:,:,::1] np_scores, int max_np):

#     # error, don't allow
#     if ref_np_len <= 0:
#         return 100
#     elif ref_np_len + indel_len < 0:
#         return 100
#     elif n < 1 or n > max_np:
#         return 100

#     # force lengths to fit in matrix
#     cdef int call_np_len = ref_np_len + indel_len
#     if ref_np_len > max_np-1:
#         ref_np_len = max_np-1
#     if call_np_len > max_np-1:
#         call_np_len = max_np-1

#     return np_scores[n-1, ref_np_len, call_np_len]


# @cython.boundscheck(False)
# @cython.wraparound(False)
# cdef int[::1] get_inss(str cigar):
#     ''' CIGAR must contain only "I" and "D". '''

#     cdef int cig_len = len(cigar)
#     inss_buf = numpy.zeros(cig_len+1, dtype=numpy.intc)
#     cdef int[::1] inss = inss_buf
#     cdef int i

#     for i in range(cig_len):
#         if cigar[i] == 'I':
#             inss[i+1] = inss[i] + 1
#         else:
#             inss[i+1] = inss[i]
#     return inss



# @cython.boundscheck(False)
# @cython.wraparound(False)
# cdef int[::1] get_dels(str cigar):
#     ''' CIGAR must contain only "I" and "D". '''

#     cdef int cig_len = len(cigar)
#     dels_buf = numpy.zeros(cig_len+1, dtype=numpy.intc)
#     cdef int[::1] dels = dels_buf
#     cdef int i

#     for i in range(cig_len):
#         if cigar[i] == 'D':
#             dels[i+1] = dels[i] + 1
#         else:
#             dels[i+1] = dels[i]
#     return dels



# @cython.boundscheck(False)
# @cython.wraparound(False)
# cdef int a_to_b_row(int a_row, int a_col, int[::1] inss, int[::1] dels, int r):
#     return a_row + a_col

# @cython.boundscheck(False)
# @cython.wraparound(False)
# cdef int a_to_b_col(int a_row, int a_col, int[::1] inss, int[::1] dels, int r):
#     cdef int b_row, b_col
#     b_row = a_row + a_col
#     b_col = inss[b_row] - a_row + r
#     return b_col



# @cython.boundscheck(False)
# @cython.wraparound(False)
# cdef int b_to_a_row(int b_row, int b_col, int[::1] inss, int[::1] dels, int r):
#     return inss[b_row] + r - b_col

# @cython.boundscheck(False)
# @cython.wraparound(False)
# cdef int b_to_a_col(int b_row, int b_col, int[::1] inss, int[::1] dels, int r):
#     return dels[b_row] - r + b_col



# @cython.boundscheck(False)
# @cython.wraparound(False)
# cdef int[::1] get_breaks(int chunk_size, int array_size, int[::1] inss, int[::1] dels):
#     cdef int buf_len = 1 + math.ceil( (array_size-1) / (chunk_size-1) )
#     breaks_buf = numpy.zeros(buf_len, dtype=numpy.intc)
#     cdef int[::1] breaks = breaks_buf
#     cdef int i
#     for i in range(buf_len-1):
#         breaks[i] = i * (chunk_size-1)

#         # don't split on 'DI', since that may have originally been '='
#         if i > 0 and inss[breaks[i]+1] == inss[breaks[i]]+1 and \
#                 dels[breaks[i]] == dels[breaks[i]-1]+1:
#             breaks[i] -= 1

#     breaks[buf_len-1] = array_size-1
#     return breaks



# @cython.boundscheck(False)
# @cython.wraparound(False)
# cdef int match(char[::1] A, char[::1] B):
#     cdef int i
#     if len(A) != len(B): 
#         return 0
#     else:
#         for i in range(len(A)):
#             if A[i] != B[i]:
#                 return 0
#     return 1


# @cython.boundscheck(False)
# @cython.wraparound(False)
# @cython.cdivision(True)
# def align(char[::1] full_ref, char[::1] seq, str cigar, 
#         float[:,::1] sub_scores, float[:,:,::1] np_scores,
#         float indel_start=5, float indel_extend=1, int max_b_rows = 20000,
#         int r = 30, int verbose=0):
#     ''' Perform alignment.  '''

#     # convert CIGAR so that each movement is row+1 or col+1, enables easy banding
#     cigar = cigar.replace('X','DI').replace('=','DI').replace('M','DI')

#     # precompute offsets, breakpoints, and homopolymers
#     cdef int[::1] inss = get_inss(cigar)
#     cdef int[::1] dels = get_dels(cigar)
#     cdef int[::1] breaks = get_breaks(max_b_rows, len(seq) + len(full_ref) + 1, inss, dels)
#     cdef int[:,::1] np_info
#     cdef char[::1] ref

#     # define useful constants
#     cdef int a_rows = len(seq) + 1
#     cdef int a_cols = len(full_ref) + 1
#     cdef int b_cols = 2*r + 1
#     cdef int b_rows = -1

#     # n-polymer info indices
#     cdef int L = 0
#     cdef int L_IDX = 1
#     cdef int N = 2
#     cdef int N_IDX = 3

#     cdef int dims = 3 # dimensions in matrix
#     cdef int VAL = 0  # value (alignment score)
#     cdef int TYP = 1  # type (predecessor matrix dimension)
#     cdef int RUN = 2  # run length (for indels)

#     cdef int typs = 5 # types
#     cdef int MAT = 0  # match/substitution
#     cdef int INS = 1  # insertion
#     cdef int NPI = 2  # lengthen homopolymer
#     cdef int DEL = 3  # deletion
#     cdef int NPD = 4  # shorten homopolymer

#     cdef str aln = ''
#     cdef str full_aln = ''
#     cdef str op, s
#     # for alignment offset purposes, M -> DI. Breaks are sometimes shifted by 1, 
#     # so that this DI doesn't cross the boundary of alignment chunks. 
#     # As a result, we must increase the matrix buffer size. (hence max_b_rows+1)
#     matrix_buf = numpy.zeros((typs, max_b_rows+1, b_cols, dims), dtype=numpy.float32)
#     cdef float[:,:,:,::1] matrix = matrix_buf
#     # max single-move penalty is < 10, so a path with this penalty will never 
#     # be chosen. Still kept small enough so that len(seq)*INF < INT_MAX
#     cdef int INF = 100

#     cdef int b_row, b_col, a_row, a_col, ref_idx, seq_idx, row, col, a_row0, a_col0
#     cdef int b_top_row, b_top_col, b_left_row, b_left_col, b_diag_row, b_diag_col
#     cdef int b_runleft_row, b_runleft_col, b_runup_row, b_runup_col
#     cdef int b_ndown_row, b_ndown_col, b_nright_row, b_nright_col
#     cdef int start_row, end_row, start_col, end_col
#     cdef int run, typ, i, brk, next_brk, brk_idx
#     cdef int prev_del, prev_prev_del
#     cdef int l, l_idx, n, n_idx
#     cdef int max_np = 10
#     cdef int max_np_len = 100
#     cdef float val1, val2

#     # iterate over b matrix in chunks set by breakpoints
#     for brk_idx in range(len(breaks)-1):

#         brk = breaks[brk_idx]
#         next_brk = breaks[brk_idx+1]
#         start_row = inss[brk]
#         end_row = inss[next_brk]
#         start_col = dels[brk]
#         end_col = dels[next_brk]
#         b_rows = next_brk - brk + 1
#         matrix_buf.fill(0)
#         ref = full_ref[ start_col : end_col+1 ]
#         np_info = get_np_info(ref)

#         # initialize N-polymer matrices with invalid states
#         a_row0 = start_row + r
#         a_col0 = start_col - r
#         for b_row in range(b_rows):
#             a_row = a_row0
#             a_col = a_col0
#             for b_col in range(b_cols):
#                 if a_row >= start_row and a_col >= start_col and \
#                         a_row <= end_row and a_col <= end_col and \
#                         b_col > 0 and b_col < 2*r:
#                     matrix[NPI, b_row, b_col, VAL] = INF * (a_row-start_row + a_col-start_col)
#                     matrix[NPI, b_row, b_col, TYP] = MAT
#                     matrix[NPI, b_row, b_col, RUN] = 0
#                     matrix[NPD, b_row, b_col, VAL] = INF * (a_row-start_row + a_col-start_col)
#                     matrix[NPD, b_row, b_col, TYP] = MAT
#                     matrix[NPD, b_row, b_col, RUN] = 0
                
#                 # update pointer
#                 a_row -= 1
#                 a_col += 1
                
#             # update start pointer based on movement
#             if cigar[b_row+brk] == 'D':
#                 a_col0 += 1
#             else:
#                 a_row0 += 1

#         # calculate matrix chunk
#         a_row0 = start_row + r
#         a_col0 = start_col - r
#         for b_row in range(b_rows):
#             a_row = a_row0
#             a_col = a_col0
            
#             if b_row+brk > 0:
#                 prev_del = (cigar[b_row+brk-1] == 'D')
#             else:
#                 prev_del = False
#             if b_row+brk > 1:
#                 prev_prev_del = (cigar[b_row+brk-2] == 'D')
#             else:
#                 prev_prev_del = False
            
#             for b_col in range(b_cols):

#                 # precompute useful positions
#                 b_top_row = b_row - 1
#                 b_top_col = b_col + prev_del
#                 b_left_row = b_row - 1
#                 b_left_col = b_col - 1 + prev_del
#                 b_diag_row = b_row - 2
#                 b_diag_col = b_col - 1 + prev_del + prev_prev_del
#                 ref_idx = a_col - start_col - 1
#                 seq_idx = a_row - 1

#                 # skip cells out of range of this chunk of original "A" matrix
#                 if a_row < start_row or a_col < start_col or \
#                         a_row > end_row or a_col > end_col:
#                     # update pointer
#                     a_row -= 1
#                     a_col += 1
#                     continue

#                 # enforce new path remains within r cells of original path
#                 elif b_col == 0 or b_col == 2*r:
#                     for typ in range(typs):
#                         matrix[typ, b_row, b_col, VAL] = INF * (b_row+1)
#                         matrix[typ, b_row, b_col, TYP] = MAT
#                         matrix[typ, b_row, b_col, RUN] = 0
#                     # update pointer
#                     a_row -= 1
#                     a_col += 1
#                     continue

#                 # get n-polymer info
#                 if a_col >= a_cols - 1:
#                     l = l_idx = n = n_idx = 0
#                 else:
#                     l     = np_info[L, ref_idx+1]
#                     l_idx = np_info[L_IDX, ref_idx+1]
#                     n     = np_info[N, ref_idx+1]
#                     n_idx = np_info[N_IDX, ref_idx+1]


#                 # UPDATE INS MATRIX                                              
#                 if a_row == start_row: # first row                               
#                     matrix[INS, b_row, b_col, VAL] = INF * (a_col-start_col+1)   
#                     matrix[INS, b_row, b_col, TYP] = DEL                         
#                     matrix[INS, b_row, b_col, RUN] = a_col - start_col           
#                 else:                                                            
#                     val1 = matrix[MAT, b_top_row, b_top_col, VAL] + indel_start  
#                     matrix[INS, b_row, b_col, VAL] = val1                        
#                     matrix[INS, b_row, b_col, TYP] = INS                         
#                     matrix[INS, b_row, b_col, RUN] = 1                           
                                                                                 
#                     val2 = matrix[INS, b_top_row, b_top_col, VAL] + indel_extend 
#                     if val2 < val1:                                              
#                         if a_row == start_row + 1:                               
#                             run = 1                                              
#                         else:                                                    
#                             run = <int>(matrix[INS, b_top_row, b_top_col, RUN]) + 1
#                         matrix[INS, b_row, b_col, VAL] = val2                    
#                         matrix[INS, b_row, b_col, TYP] = INS                     
#                         matrix[INS, b_row, b_col, RUN] = run                     

                        
#                 # UPDATE DEL MATRIX                                              
#                 if a_col == dels[brk]: # first col                               
#                     matrix[DEL, b_row, b_col, VAL] = INF * (a_row-start_row+1)   
#                     matrix[DEL, b_row, b_col, TYP] = INS                         
#                     matrix[DEL, b_row, b_col, RUN] = a_row - start_row           
#                 else:                                                            
#                     val1 = matrix[MAT, b_left_row, b_left_col, VAL] + indel_start
#                     matrix[DEL, b_row, b_col, VAL] = val1                        
#                     matrix[DEL, b_row, b_col, TYP] = DEL                         
#                     matrix[DEL, b_row, b_col, RUN] = 1                           
                                                                                 
#                     val2 = matrix[DEL, b_left_row, b_left_col, VAL] + indel_extend
#                     if val2 < val1:                                              
#                         if a_col == start_col + 1:                               
#                             run = 1                                              
#                         else:                                                    
#                             run = <int>(matrix[DEL, b_left_row, b_left_col, RUN]) + 1
#                         matrix[DEL, b_row, b_col, VAL] = val2                    
#                         matrix[DEL, b_row, b_col, TYP] = DEL                     
#                         matrix[DEL, b_row, b_col, RUN] = run                     


#                 # UPDATE MAT MATRIX
#                 if a_row > start_row and a_col > start_col: # can move diag
#                     if matrix[MAT, b_diag_row, b_diag_col, TYP] == MAT:
#                         run = <int>(matrix[MAT, b_diag_row, b_diag_col, RUN]) + 1
#                     else:
#                         run = 1
#                     val1 = matrix[MAT, b_diag_row, b_diag_col, VAL] + \
#                             sub_scores[ seq[seq_idx], ref[ref_idx] ]
#                     matrix[MAT, b_row, b_col, VAL] = val1
#                     matrix[MAT, b_row, b_col, TYP] = MAT
#                     matrix[MAT, b_row, b_col, RUN] = run

#                 else:
#                     # ensure val1 isn't chosen
#                     val1 = matrix[DEL, b_row, b_col, VAL] + INF

#                 # end INDEL
#                 for typ in range(1,typs): # [INS, NPI, DEL, NPD]
#                     val2 = matrix[typ, b_row, b_col, VAL]
#                     if val2 < val1:
#                         val1 = val2
#                         run = <int>(matrix[typ, b_row, b_col, RUN])
#                         matrix[MAT, b_row, b_col, VAL] = val2
#                         matrix[MAT, b_row, b_col, TYP] = typ
#                         matrix[MAT, b_row, b_col, RUN] = run


#                 # UPDATE NPI MATRIX
#                 if a_row == start_row: # first row
#                     matrix[NPI, b_row, b_col, VAL] = INF * (a_col-start_col)
#                     matrix[NPI, b_row, b_col, TYP] = DEL
#                     matrix[NPI, b_row, b_col, RUN] = a_col - start_col

#                 # start insertion
#                 b_ndown_row = b_row + n
#                 b_ndown_col = a_to_b_col(a_row+n, a_col, inss, dels, r)
#                 if a_row+n <= end_row and b_ndown_col > 0: # np stays in chunk
#                     if n > 0 and n_idx == 0 and l_idx == 0 and \
#                             match(ref[ref_idx+1:ref_idx+n+1], seq[seq_idx+1:seq_idx+1+n]):
#                         val1 = matrix[MAT, b_row, b_col, VAL] + \
#                                 np_score(n, l, 1, np_scores, max_np_len)
#                         matrix[NPI, b_ndown_row, b_ndown_col, VAL] = val1
#                         matrix[NPI, b_ndown_row, b_ndown_col, TYP] = NPI
#                         matrix[NPI, b_ndown_row, b_ndown_col, RUN] = n
                            
#                     # continue insertion
#                     elif n > 0 and n_idx == 0 and \
#                             match(ref[ref_idx+1:ref_idx+n+1], seq[seq_idx:seq_idx+n]): 
#                         run = <int>(matrix[NPI, b_row, b_col, RUN]) + n
#                         b_runup_row = a_to_b_row(a_row+n-run, a_col, inss, dels, r) - brk
#                         b_runup_col = a_to_b_col(a_row+n-run, a_col, inss, dels, r)
#                         if run > n and a_row+n-run >= start_row and b_runup_col < 2*r:
#                             val2 = matrix[MAT, b_runup_row, b_runup_col, VAL] + \
#                                 np_score(n, l, <int>(run/n), np_scores, max_np_len)
#                             matrix[NPI, b_ndown_row, b_ndown_col, VAL] = val2
#                             matrix[NPI, b_ndown_row, b_ndown_col, TYP] = NPI
#                             matrix[NPI, b_ndown_row, b_ndown_col, RUN] = run


#                 # UPDATE NPD MATRIX
#                 if a_col == start_col: # first col
#                     matrix[NPD, b_row, b_col, VAL] = INF * (a_row-start_row)
#                     matrix[NPD, b_row, b_col, TYP] = INS
#                     matrix[NPD, b_row, b_col, RUN] = a_row - start_row

#                 b_nright_row = b_row + n
#                 b_nright_col = a_to_b_col(a_row, a_col+n, inss, dels, r)
#                 if a_col+n <= end_col and b_nright_col < 2*r: # np spans breakpoint
#                     if n > 0 and n_idx == 0 and l_idx == 0: # start deletion
#                         val1 = matrix[MAT, b_row, b_col, VAL] + \
#                             np_score(n, l, -1, np_scores, max_np_len)
#                         matrix[NPD, b_nright_row, b_nright_col, VAL] = val1
#                         matrix[NPD, b_nright_row, b_nright_col, TYP] = NPD
#                         matrix[NPD, b_nright_row, b_nright_col, RUN] = n

#                     elif n > 0 and n_idx == 0: # continue deletion
#                         run = <int>(matrix[NPD, b_row, b_col, RUN]) + n
#                         b_runleft_row = a_to_b_row(a_row, a_col+n-run, inss, dels, r) - brk
#                         b_runleft_col = a_to_b_col(a_row, a_col+n-run, inss, dels, r)
#                         if run > n and a_col+n-run >= start_col and b_runleft_col > 0:
#                             val2 = matrix[MAT, b_runleft_row, b_runleft_col, VAL] + \
#                                 np_score(n, l, <int>(-run/n), np_scores, max_np_len)
#                             matrix[NPD, b_nright_row, b_nright_col, VAL] = val2
#                             matrix[NPD, b_nright_row, b_nright_col, TYP] = NPD
#                             matrix[NPD, b_nright_row, b_nright_col, RUN] = run
                
#                 # update pointer
#                 a_row -= 1
#                 a_col += 1
                
#             # update start pointer based on movement
#             if cigar[b_row+brk] == 'D':
#                 a_col0 += 1
#             else:
#                 a_row0 += 1
                
#         # initialize backtracking from last cell
#         a_row = end_row
#         a_col = end_col
#         aln = ''
#         b_row = a_to_b_row(a_row, a_col, inss, dels, r) - brk
#         b_col = a_to_b_col(a_row, a_col, inss, dels, r)
#         typ = <int>(matrix[MAT, b_row, b_col, VAL])
#         run = 0
#         path = []

#         # backtrack
#         while a_row > start_row or a_col > start_col:
#             b_row = a_to_b_row(a_row, a_col, inss, dels, r) - brk
#             b_col = a_to_b_col(a_row, a_col, inss, dels, r)
#             val = matrix[MAT, b_row, b_col, VAL]
#             typ = <int>(matrix[MAT, b_row, b_col, TYP])
#             run = <int>(matrix[MAT, b_row, b_col, RUN])

#             # if verbose: # do some error-checking
#             path.append((MAT, a_row, a_col))
#             if a_row < 0:
#                 print(f"ERROR: row < 0 @ A:({a_row},{a_col}), B:({b_row},{b_col})")
#                 break
#             if a_col < 0:
#                 print(f"ERROR: col < 0 @ A:({a_row},{a_col}), B:({b_row},{b_col})")
#                 break
#             if run < 1:
#                 print(f"\nERROR: run 0 @ A:({a_row},{a_col}), B:({b_row},{b_col}),  type {typ}, val {val}")
#                 break

#             op = ''
#             if typ == NPI or typ == INS:   # each move is an insertion
#                 for i in range(run):
#                     op += 'I' #if typ == INS else 'L'
#                 a_row -= run
#             elif typ == NPD or typ == DEL: # each move is a deletion
#                 for i in range(run):
#                     op += 'D' #if typ == DEL else 'S'
#                 a_col -= run
#             elif typ == MAT: # only sub if stay in same matrix
#                 i = 0
#                 while i < run:
#                     a_row -= 1
#                     a_col -= 1
#                     if ref[a_col-start_col] == seq[a_row]:
#                         op += '='
#                     else:
#                         op += 'X'
#                     i += 1
#             else:
#                 print("ERROR: unknown alignment matrix type:", typ)
#                 break
#             aln += op

#         full_aln += aln[::-1]

#     return full_aln

In [ ]:
base_dict = {'N':0, 'A':1, 'C':2, 'G':3, 'T':4}
subs_fn = "../guppy5_stats/subs_cm.npy"
nps_fn = "../guppy5_stats/nps_cm.npy"
inss_fn = "../guppy5_stats/inss_cm.npy"
dels_fn = "../guppy5_stats/dels_cm.npy"
subs = np.load(subs_fn)
nps = np.load(nps_fn)
dels = np.load(dels_fn)
inss = np.load(inss_fn)
sub_scores, np_scores, ins_scores, del_scores = \
        calc_score_matrices(subs, nps, inss, dels)

read_file = open("read0.csv", "r")
ref, seq, cig = read_file.readline().split()

int_ref = np.zeros(len(ref), dtype=np.uint8)                                                                                                                                                                                          
for i in range(len(ref)):                                                                                                                                                                                                             
    int_ref[i] = base_dict[ref[i]]                                                                                                                                                                                                
int_seq = np.zeros(len(seq), dtype=np.uint8)                                                                                                                                                                                              
for i in range(len(seq)):                                                                                                                                                                                                                 
    int_seq[i] = base_dict[seq[i]] 

In [ ]:
# lp = line_profiler.LineProfiler()
# lp.add_function(align)
# lp.runcall(align, int_ref, int_seq, cig, sub_scores, np_scores)
# lp.print_stats()

In [ ]:
from time import perf_counter
times = []
for x in range(10):
    print('.', end='')
    start = perf_counter()
    a = align(int_ref, int_seq, cig, sub_scores, np_scores)
    times.append(perf_counter()-start)
print(' ')
print(np.median(times))

In [ ]:
# with open('aln.txt', 'w') as f:
#     f.write(a)

In [ ]:
a = align(int_ref, int_seq, cig, sub_scores, np_scores)

In [ ]:
with open('aln.txt', 'r') as f:
    aln = f.readline()
print(a == aln)